In [1]:
#from random import shuffle
from math import isnan, sqrt
import igraph

# Section 2

In [2]:
graph = igraph.Graph(directed=True)
vertices = set()
edges = []

with open("shared/ecolitfnet.txt", "r") as f:
    for line in f:
        edges.append(tuple(line.rstrip().split()))
            
graph = igraph.Graph.TupleList(edges=edges, directed=True)
print(graph.summary())

IGRAPH DN-- 133 261 -- 
+ attr: name (v)


# Section 3

![Graph motifs](https://6yn6ig.bl3302.livefilestore.com/y3myI__C7wk1XA3nLx6IY4CNe1D4jO49sAN1xGSaYFsp_Q6DqA-rOKk812xzdqLW1xjjV9GKm7iyEJfir5ZnS1ocO9xgjJNW4_gYWLcYzySRhbWdclp1ID-gPrqD9RUe-cbnIyxo2mnsB7I3X0Wjpk541fHepCIIMJazliMTqppTzQ/igraph_motifs.png?psid=1)

In [3]:
for pair in zip(range(16), graph.motifs_randesu()):
    if isnan(pair[1]):
        continue
    print("Motif", pair[0], "occurs", pair[1], "times in the graph.")

Motif 2 occurs 275 times in the graph.
Motif 4 occurs 125 times in the graph.
Motif 5 occurs 39 times in the graph.
Motif 6 occurs 1676 times in the graph.
Motif 7 occurs 47 times in the graph.
Motif 8 occurs 25 times in the graph.
Motif 9 occurs 68 times in the graph.
Motif 10 occurs 0 times in the graph.
Motif 11 occurs 0 times in the graph.
Motif 12 occurs 2 times in the graph.
Motif 13 occurs 10 times in the graph.
Motif 14 occurs 1 times in the graph.
Motif 15 occurs 1 times in the graph.


**Motif 6** appears the most times in the graph, with _1,676 appearances_.

The motif with _47 appearances_ (the Mystery Motif) is **motif 7**, the forward feedback loop. 

In [4]:
motif_occurences = []
random_graph = graph.copy()

#sources, targets = zip(*edges) # This syntax unzips a list of tuples
#sources, targets = list(sources), list(targets) # Convert tuples to lists
for i in range(10000):
    #shuffle(targets)
    #random_graph = igraph.Graph.TupleList(edges=list(zip(sources, targets)), directed=True)
    random_graph.rewire() # Replaced the shuffling technique with rewire(), which prevents self-loops
    motif_occurences.append(random_graph.motifs_randesu()[7])
    

In [13]:
mean = sum(motif_occurences)/len(motif_occurences)
standard_deviation = sqrt(sum(map(lambda x: (x - mean)**2, motif_occurences))/len(motif_occurences))
z_score = (47 - mean) / standard_deviation

print("Mystery Motif in random networks:")
print("    Mean:", mean)
print("    Standard deviation:", standard_deviation)
print("    Z-score:", z_score) # Sure, this looks statistically significant. It's far past the standard deviation
print("    Ratio of real value to mean:", 47/mean)

Mystery Motif in random networks:
    Mean: 57.2378
    Standard deviation: 9.251629648878104
    Z-score: -1.106594231346202
    Ratio of real value to mean: 0.8211356830625915


**Question:** How do these values compare to Table 1 of Reading 18 in Shen-Orr et al., Nature Genetics, 2002?

**My answer:** Shen-Orr et al. measured 40 feed-foward loops in their real network and 7 in the randomized networks, with a ratio of 5.714. My measured values are significantly different then theirs; I measured slightly more feed-forward loops in the random networks and they measured significantly less in the randomized networks. 

**Question:** Given the modest ratio of the MM frequency in the real network vs. randomly
shuffled network, should we entertain the possibility that the high frequency of
MMs in the real network could be a conseqeuence of the degree distribution
rather than evolution specifically “favoring” FFLs as a building block for gene
regulatory networks?

**My answer:** Given that in random networks, the mystery motif appears on average more than one whole standard deviation more than the actual network, I think that's that's strong evidence that the feed-forward loop just appears as an artifact of a network with this degree distribution. 

# Section 4

In [6]:
# This code snipped will iterate and print out each class with its configuration.
# It is commented out to keep the output brief.
#for i in range(218):
#    g = igraph.Graph.Isoclass(4, i, directed=True)
#    if g.ecount() == 4:
#        print("\nClass", i, "\n", igraph.Graph.Isoclass(4, i, directed=True))

# We desire a class 19 subgraph
print("Class 19 subgraph:")
print(igraph.Graph.Isoclass(4, 19, directed=True))
print("\nThere are a total of", graph.motifs_randesu(4)[19], "total occurences of this motif in the E. coli gene regulatory netowork.")

Class 19 subgraph:
IGRAPH D--- 4 4 --
+ edges:
3->1 2->1 3->0 2->0

There are a total of 297 total occurences of this motif in the E. coli gene regulatory netowork.


In [7]:
quad_motif_occurences = []

for i in range(10000):
    #shuffle(targets)
    #random_graph = igraph.Graph.TupleList(edges=list(zip(sources, targets)), directed=True)
    random_graph.rewire()
    quad_motif_occurences.append(random_graph.motifs_randesu(4)[19])
    
mean = sum(quad_motif_occurences)/len(quad_motif_occurences)
standard_deviation = sqrt(sum(map(lambda x: (x - mean)**2, quad_motif_occurences))/len(quad_motif_occurences))
z_score = (297 - mean) / standard_deviation

print("Four-vertex motif analysis:")
print("    Mean:", mean)
print("    Standard deviation:", standard_deviation)
print("    Z-score:", z_score)
print("    Ratio of real value to mean:", 297/mean) # TODO: How does this compare to Table 1 of Reading 18?

Four-vertex motif analysis:
    Mean: 177.4322
    Standard deviation: 38.51979754827441
    Z-score: 3.1040609663161725
    Ratio of real value to mean: 1.673878811174071


**Question:** How does this ratio compare to the ratio that you obtain
from Table 1 from Shen-Orr et al.? Are they consistent?

**My Answer:** Shen-Orr et al. reported 203 ocurrences of this motif in the real network 57 in the random network. That means a ratio of 3.561. Although we have both calculated that this motif occurs in the real network more times than in the random network, Shen-Orr et al. calculated a dramatically higher ratio then I was able to. 

**Question:** Does this suggest that
Shen-Orr’s actual network randomization procedure is possibly not consistent
with their description in the Methods section of their paper, i.e., that it may
have had some kind of error?

**My answer:** Yes, I believe that their techniques may have been prone to error, given that I was unable to reproduce their results. I have either calculated completely different results or results that are far less extreme then theirs. 

# Extra Credit

In [8]:
graph = igraph.Graph.Barabasi(20, 2, directed=True)
print(graph.summary())

IGRAPH D--- 20 37 -- 


In [9]:
motifs = {}
vertex_sets = set()

for first in graph.vs:    
    for second in first.neighbors():
        for third in second.neighbors():
            if third in (first, second):
                continue # Ignore graphs with only two vertices
            
            vertex_set = frozenset((first, second, third))
            if vertex_set in vertex_sets:
                continue # Ignore graphs if we've already seen these three vertices before
            else:
                vertex_sets.add(vertex_set)
                
            g = graph.subgraph([first, second, third])
            #print(g.summary())
            motif = frozenset({e.tuple for e in g.es})
            #print(motif)
            motifs[motif] = motifs.get(motif, 0) + 1
            
print("Network motifs:")
for motif in motifs:
    print("   ", str(motif)[11:-2].replace("),", "**").replace(", ", " -> ").replace("**", "),"), "occurs", motifs[motif], "times in this graph.")
print("    All other triple motifs appear 0 times in this graph.")

Network motifs:
    (2 -> 0), (1 -> 0), (2 -> 1) occurs 9 times in this graph.
    (1 -> 0), (2 -> 1) occurs 44 times in this graph.
    (2 -> 0), (2 -> 1) occurs 9 times in this graph.
    (2 -> 0), (1 -> 0) occurs 65 times in this graph.
    All other triple motifs appear 0 times in this graph.


In [10]:
print("Actual number of motifs:", graph.motifs_randesu())

Actual number of motifs: [nan, nan, 65, nan, 44, 0, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0]
